In [1]:


import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [6]:
# AWS와 PM 데이터프레임 로드
aws_df = pd.read_csv("AWS_TRAIN.csv")
pm_df  = pd.read_csv("PM_TRAIN.csv")
aws_df
aws_df_missing_values = aws_df.isnull().sum()
print(aws_df_missing_values)


연도            0
일시            0
지점            0
기온(°C)        0
풍향(deg)       0
             ..
기온(°C).29     0
풍향(deg).29    0
풍속(m/s).29    0
강수량(mm).29    0
습도(%).29      0
Length: 182, dtype: int64


In [112]:
df_no_missing = df.dropna()

features = ['기온(°C)', '풍향(deg)', '풍속(m/s)', '강수량(mm)', '습도(%)']
target = features

X_train = df_no_missing[features]
y_train = df_no_missing[target]


In [113]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

def create_sequences(data, seq_len):
    x, y = [], []
    for i in range(len(data) - seq_len):
        x.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return np.array(x), np.array(y)

seq_len = 5
x_train, y_train = create_sequences(X_train, seq_len)


In [117]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], len(features))),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(len(features))
])

model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(x_train, y_train, epochs=15, batch_size=10, verbose=2)

Epoch 1/15
3400/3400 - 33s - loss: 0.0167 - 33s/epoch - 10ms/step
Epoch 2/15
3400/3400 - 32s - loss: 0.0140 - 32s/epoch - 9ms/step
Epoch 3/15
3400/3400 - 31s - loss: 0.0136 - 31s/epoch - 9ms/step
Epoch 4/15
3400/3400 - 32s - loss: 0.0134 - 32s/epoch - 9ms/step
Epoch 5/15
3400/3400 - 31s - loss: 0.0133 - 31s/epoch - 9ms/step
Epoch 6/15
3400/3400 - 31s - loss: 0.0132 - 31s/epoch - 9ms/step
Epoch 7/15
3400/3400 - 31s - loss: 0.0131 - 31s/epoch - 9ms/step
Epoch 8/15
3400/3400 - 30s - loss: 0.0130 - 30s/epoch - 9ms/step
Epoch 9/15
3400/3400 - 30s - loss: 0.0130 - 30s/epoch - 9ms/step
Epoch 10/15
3400/3400 - 30s - loss: 0.0129 - 30s/epoch - 9ms/step
Epoch 11/15
3400/3400 - 30s - loss: 0.0129 - 30s/epoch - 9ms/step
Epoch 12/15
3400/3400 - 30s - loss: 0.0129 - 30s/epoch - 9ms/step
Epoch 13/15
3400/3400 - 30s - loss: 0.0128 - 30s/epoch - 9ms/step
Epoch 14/15
3400/3400 - 30s - loss: 0.0128 - 30s/epoch - 9ms/step
Epoch 15/15
3400/3400 - 30s - loss: 0.0128 - 30s/epoch - 9ms/step


In [118]:
missing_rows = df[df.isna().any(axis=1)]

for idx, row in missing_rows.iterrows():
    input_data = df.loc[idx - seq_len: idx - 1, features]
    input_data = scaler.transform(input_data)
    input_data = np.expand_dims(input_data, axis=0)
    prediction = model.predict(input_data)
    prediction = scaler.inverse_transform(prediction)

    df.loc[idx, features] = prediction

1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 10ms/step


In [119]:
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 3
X, Y = create_dataset(target, look_back)
print(Y)
print(X)


TypeError: list indices must be integers or slices, not tuple

In [69]:
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))


In [70]:
model = Sequential()
model.add(LSTM(50, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(X, Y, epochs=10, batch_size=10, verbose=2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Timestamp).

In [27]:

missing_indexes = target[target['풍향(deg)'].isna()].index
for i in missing_indexes:
    input_data = target_scaled[i - look_back:i].reshape(1, 1, look_back)
    pred = model.predict(input_data)
    target.loc[i, '풍향(deg)'] = scaler.inverse_transform(pred)[0, 0]


Epoch 1/100


2023-05-04 14:45:39.619556: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


33973/33973 - 143s - loss: 0.0668 - 143s/epoch - 4ms/step
Epoch 2/100


KeyboardInterrupt: 